In [4]:
import fitz  # PyMuPDF
import os

def extract_text_from_pdfs(pdf_folder):
    documents = []
    
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            doc_path = os.path.join(pdf_folder, file)
            doc = fitz.open(doc_path)

            text = ""
            for page in doc:
                text += page.get_text("text") + "\n"
            
            documents.append({"filename": file, "content": text})
    
    return documents

# Exemplo de uso
pdf_folder = "C:/Users/Administrador/Downloads/Procurement-_AI_Agent/data"
pdf_data = extract_text_from_pdfs(pdf_folder)

# Exibir o primeiro artigo extraído
print(pdf_data[0]["content"][:1000])  # Mostra os primeiros 1000 caracteres


See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/326790494
ANÁLISE DE RISCO E CLASSIFICAÇÃO DE FORNECEDORES EM UMA CADEIA
DE SUPRIMENTOS
Article  in  Engevista · November 2012
DOI: 10.22409/engevista.v15i2.461
CITATIONS
3
READS
1,196
5 authors, including:
Miguel Afonso Sellitto
Universidade do Vale do Rio dos Sinos, Brazil
304 PUBLICATIONS   3,880 CITATIONS   
SEE PROFILE
Miriam Borchardt
Universidade do Vale do Rio dos Sinos
156 PUBLICATIONS   2,411 CITATIONS   
SEE PROFILE
Giancarlo Medeiros Pereira
Universidade do Vale do Rio dos Sinos
137 PUBLICATIONS   2,286 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Miguel Afonso Sellitto on 23 August 2018.
The user has requested enhancement of the downloaded file.

 

 






	




	

	

	





















ANÁLISE DE RISCO E CLASSIFICAÇÃO DE FORNECEDORES EM UMA 
CADEIA DE SUPRIMENTOS
Miguel Afonso Sellitto1
Bruna Omizzo


In [11]:
len(pdf_data[0]["content"])

50571

In [14]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Criar um cliente do ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Definir a função de embedding (modelo de NLP para transformar texto em vetor)
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Criar coleção
collection = chroma_client.get_or_create_collection(name="procurement_articles", embedding_function=embedding_function)

# Adicionar os artigos processados
for idx, doc in enumerate(pdf_data):
    collection.add(
        ids=[str(idx)], 
        documents=[doc["content"]],
        metadatas=[{"filename": doc["filename"]}]
    )

print("Artigos indexados com sucesso! ✅")

print(type(collection))

c:\Users\Administrador\Downloads\Procurement-_AI_Agent\ambiente_virtual\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Administrador\Downloads\Procurement-_AI_Agent\ambiente_virtual\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrador\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you eithe

Artigos indexados com sucesso! ✅
<class 'chromadb.api.models.Collection.Collection'>


In [18]:
def search_articles(query, top_k=3):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    return results["documents"]

# Exemplo de busca
query = "Quais são os principais desafios estratégicos no procurement?"
context = search_articles(query)
print("\n".join(context[0]))  # Exibe o artigo mais relevante


 
MAURICIO BERTOLDI SANCHEZ NEVES 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
ANÁLISE DOS DIFERENTES MODELOS DE ESTRUTURAÇÃO DE 
PROCUREMENT EM EMPRESAS DE REAL ESTATE 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
São Paulo 
2016 

 
MAURICIO BERTOLDI SANCHEZ NEVES 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
ANÁLISE DOS DIFERENTES MODELOS DE ESTRUTURAÇÃO DE 
PROCUREMENT EM EMPRESAS DE REAL ESTATE 
 
 
 
 
 
 
Dissertação apresentada à Escola Politécnica 
da Universidade de São Paulo para obtenção 
do título de Mestre em Ciências. 
 
Área de Concentração: Engenharia de 
Construção Civil e Urbana 
Real Estate 
 
Orientador:  
Prof. Dr. João da Rocha Lima Jr. 
 
 
 
 
 
 
 
 
 
 
 
São Paulo 
2016 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Catalogação-na-publicação 
 
 
 
 
 
Neves, Mauricio 
ANÁLISE DOS DIFERENTES MODELOS DE ESTRUTURAÇÃO DE 
PROCUREMENT EM EMPRESAS DE REAL ESTATE / M. Neves -- 
versão 
corr. -- São Paulo, 2016. 
132 p. 
 
Dissertação (Mestrado) - Escola Pol

In [19]:
import ollama

def ask_llava_with_context(question):
    context = search_articles(question)
    context_str = "\n\n".join(context[0])  # Concatenar o conteúdo dos artigos

    prompt = f"""
    Baseado nos seguintes artigos sobre procurement:
    {context_str}

    Responda à seguinte pergunta:
    {question}
    """

    response = ollama.chat(model="llava", messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]

# Exemplo de uso
question = "Como podemos melhorar a eficiência na negociação com fornecedores?"
answer = ask_llava_with_context(question)
print(answer)


ResponseError: model "llava" not found, try pulling it first (status code: 404)